# Step 4a: Attention Fundamentals — The Database Query Analogy 🎯

Every word in a sentence needs context to be understood. "Bank" means something different in "river bank" vs "bank account." Attention is the mechanism that lets each token look at other tokens and gather the context it needs — think of it like each token running a database query against all other tokens to find relevant information.

## What We'll Learn

1. **The Problem** — Why isolated tokens aren't enough
2. **Q, K, V: The Database Query Analogy** — Query, Key, Value as a lookup system
3. **Scaled Dot-Product Attention** — The core algorithm step by step
4. **Softmax: Weighted Load Balancing** — Converting scores to percentages
5. **Single-Head Attention** — Putting it all together

In **Step 4b**, we'll extend this to multi-head attention and add causal masking.

---

## The Problem: Tokens in Isolation Know Nothing

After embeddings (Step 3), each token has a 768-dimensional vector. But here's the problem:

```
"The server crashed because it ran out of memory"
     ↓
Each token has its own embedding, but:
  - "it" doesn't know it refers to "server"
  - "crashed" doesn't know what crashed
  - Each embedding is computed in isolation!
```

**Think of it like this:** Imagine you're a customer support agent handling a ticket. You can see ONE word of the ticket at a time. How would you understand "Please restart it"? You need context!

**The solution: Self-Attention**

Each token gets to "query" all other tokens and gather relevant context:

```
Token "it" asks: "Who in this sentence am I referring to?"
                           ↓
   Attention scores: [server: 0.85, crashed: 0.05, memory: 0.03, ...]
                           ↓
   "it" now carries information about "server" in its representation
```

This is exactly like a **database query** — you have a query, you search against keys, and you retrieve values. Let's see how.


---

## Setup: Load Everything from Previous Steps


In [ ]:
import json
import numpy as np
import regex


In [ ]:
# Load SafeTensors (from Step 1)
def load_safetensors(path: str) -> dict:
    """Load a SafeTensors file without external library."""
    with open(path, 'rb') as f:
        header_size = int.from_bytes(f.read(8), 'little')
        header_json = f.read(header_size).decode('utf-8')
        header = json.loads(header_json)
        header.pop("__metadata__", {})
        
        tensors = {}
        data_start = 8 + header_size
        
        for name, info in header.items():
            dtype_map = {"F32": np.float32, "F16": np.float16, "I64": np.int64}
            dtype = dtype_map.get(info["dtype"], np.float32)
            offset_start, offset_end = info["data_offsets"]
            
            f.seek(data_start + offset_start)
            raw_data = f.read(offset_end - offset_start)
            tensors[name] = np.frombuffer(raw_data, dtype=dtype).reshape(info["shape"])
        
        return tensors

# Load model weights
tensors = load_safetensors("models/gpt2/model.safetensors")

# Load config
with open("models/gpt2/config.json") as f:
    config = json.load(f)

print(f"✅ Loaded {len(tensors)} weight tensors")
print(f"\nKey hyperparameters:")
print(f"  n_vocab: {config['vocab_size']:,}")
print(f"  n_embd:  {config['n_embd']}")
print(f"  n_head:  {config['n_head']}")
print(f"  n_layer: {config['n_layer']}")


In [ ]:
# GPT-2 Tokenizer (from Step 2)
def bytes_to_unicode():
    """Create byte-to-unicode mapping for GPT-2."""
    printable = list(range(ord("!"), ord("~") + 1))
    printable += list(range(ord("¡"), ord("¬") + 1))
    printable += list(range(ord("®"), ord("ÿ") + 1))
    
    byte_values = printable[:]
    unicode_chars = printable[:]
    
    n = 0
    for b in range(256):
        if b not in printable:
            byte_values.append(b)
            unicode_chars.append(256 + n)
            n += 1
    
    return {b: chr(c) for b, c in zip(byte_values, unicode_chars)}


class GPT2Tokenizer:
    """BPE tokenizer for GPT-2."""
    
    def __init__(self, vocab_path="models/gpt2/vocab.json", merges_path="models/gpt2/merges.txt"):
        with open(vocab_path, "r", encoding="utf-8") as f:
            self.vocab = json.load(f)
        self.vocab_inverse = {v: k for k, v in self.vocab.items()}
        
        with open(merges_path, "r", encoding="utf-8") as f:
            merges_text = f.read()
        merge_lines = [line for line in merges_text.split("\n")[1:] if line]
        self.bpe_merges = {}
        for i, line in enumerate(merge_lines):
            parts = line.split(" ")
            if len(parts) == 2:
                self.bpe_merges[tuple(parts)] = i
        
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        self.pattern = regex.compile(
            r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        )
        self.cache = {}
    
    def _get_pairs(self, word):
        return set((word[i], word[i + 1]) for i in range(len(word) - 1))
    
    def _bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        
        word = list(token)
        while len(word) > 1:
            pairs = self._get_pairs(word)
            if not pairs:
                break
            best_pair = min(pairs, key=lambda p: self.bpe_merges.get(p, float('inf')))
            if best_pair not in self.bpe_merges:
                break
            first, second = best_pair
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = new_word
        
        result = tuple(word)
        self.cache[token] = result
        return result
    
    def encode(self, text):
        token_ids = []
        for chunk in self.pattern.findall(text):
            byte_encoded = ''.join(self.byte_encoder[b] for b in chunk.encode('utf-8'))
            bpe_tokens = self._bpe(byte_encoded)
            for token in bpe_tokens:
                if token in self.vocab:
                    token_ids.append(self.vocab[token])
        return token_ids
    
    def decode(self, token_ids):
        text = ''.join(self.vocab_inverse[tid] for tid in token_ids)
        byte_values = [self.byte_decoder[c] for c in text]
        return bytes(byte_values).decode('utf-8', errors='replace')
    
    def get_token_string(self, token_id):
        return self.vocab_inverse.get(token_id, "<UNK>")


tokenizer = GPT2Tokenizer()
print(f"✅ Tokenizer loaded with {len(tokenizer.vocab):,} tokens")


In [ ]:
# Get embeddings (from Step 3)
wte = tensors["wte.weight"]  # Token embeddings: [50257, 768]
wpe = tensors["wpe.weight"]  # Position embeddings: [1024, 768]

def get_input_embeddings(text):
    """Get the input embeddings for text (token + position)."""
    token_ids = tokenizer.encode(text)
    seq_len = len(token_ids)
    token_emb = wte[token_ids]
    position_emb = wpe[:seq_len]
    return token_emb + position_emb, token_ids

# Test
test_emb, test_ids = get_input_embeddings("Hello world")
print(f"✅ Embedding function ready")
print(f"   'Hello world' -> shape {test_emb.shape}")


---

## Q, K, V: Attention Works Like a Database Query

If you've ever used a key-value store like Redis or queried a database, you already understand the core concept of attention!

### The Database Analogy

Think of attention as each token running a query against a database:

| Attention Term | Database Analogy | What It Does |
|----------------|------------------|--------------|
| **Query (Q)** | Your SQL SELECT query | "What am I looking for?" |
| **Key (K)** | Indexed columns to search | "Here's what each row contains" |
| **Value (V)** | The actual data in rows | "Here's the information to retrieve" |

**Concrete Example:**

Imagine a Redis-like key-value store where each token is a row:

```
┌─────────────────────────────────────────────────────────────────┐
│  Token Database                                                 │
├──────────┬─────────────────────┬───────────────────────────────┤
│  Token   │  Key (searchable)   │  Value (retrievable data)     │
├──────────┼─────────────────────┼───────────────────────────────┤
│  "The"   │  [0.2, -0.1, ...]   │  [0.5, 0.3, ...]              │
│  "server"│  [0.8, 0.3, ...]    │  [0.7, 0.1, ...]  ← relevant! │
│  "crashed"│ [0.4, 0.1, ...]    │  [0.2, 0.8, ...]              │
│  "it"    │  [0.7, 0.2, ...]    │  [0.1, 0.4, ...]              │
└──────────┴─────────────────────┴───────────────────────────────┘
```

When token "it" wants to understand its context, it:

1. **Creates a Query**: "it" generates a query vector: `Q_it = [0.75, 0.25, ...]`
2. **Searches all Keys**: Compare Q_it with each token's Key using dot product
3. **Gets relevance scores**: `{The: 0.1, server: 0.85, crashed: 0.03, it: 0.02}`
4. **Retrieves weighted Values**: Combine Values weighted by those scores

The result? Token "it" now has information about "server" mixed into its representation!

```
                    ┌──────────────────┐
     "it" Query ───►│  Search Keys     │
                    └────────┬─────────┘
                             │ Match scores
                             ▼
              ┌──────────────────────────────┐
              │  server: 0.85  ████████████  │
              │  The:    0.10  █             │
              │  crashed: 0.03               │
              │  it:     0.02                │
              └──────────────────────────────┘
                             │
                             ▼ Weighted retrieval
              ┌──────────────────────────────┐
              │  Output = 0.85 × V_server    │
              │         + 0.10 × V_The       │
              │         + 0.03 × V_crashed   │
              │         + 0.02 × V_it        │
              └──────────────────────────────┘
```

### Why Three Separate Projections?

You might ask: "Why not use the same vector for Query, Key, and Value?"

Using different projections lets the model learn:
- **What to search for** (Query) — might emphasize semantic meaning
- **What to advertise** (Key) — might emphasize grammatical role  
- **What to provide** (Value) — might emphasize the actual information content

It's like having separate indexes optimized for searching vs. the actual data you retrieve.


---

## GPT-2's Attention Weights: Under the Hood

Now let's look at how GPT-2 actually stores its attention weights. Here's a clever optimization:

**Instead of 3 separate weight matrices (W_q, W_k, W_v), GPT-2 combines them into ONE matrix!**

```
┌────────────────────────────────────────────────────────────────┐
│  c_attn.weight: [768, 2304]                                    │
│                                                                │
│  ┌──────────────┬──────────────┬──────────────┐                │
│  │   W_query    │    W_key     │   W_value    │                │
│  │  [768, 768]  │  [768, 768]  │  [768, 768]  │                │
│  └──────────────┴──────────────┴──────────────┘                │
│         ↑              ↑              ↑                        │
│    cols 0-767     cols 768-1535  cols 1536-2303                │
└────────────────────────────────────────────────────────────────┘
```

Why combine them? **Efficiency.** One matrix multiplication is faster than three separate ones. The GPU can process it in a single operation, then we split the result.

This is a common pattern in production systems: batch multiple operations together for better throughput.


In [ ]:
# Look at attention weights from the first layer
c_attn_weight = tensors["h.0.attn.c_attn.weight"]
c_attn_bias = tensors["h.0.attn.c_attn.bias"]
c_proj_weight = tensors["h.0.attn.c_proj.weight"]
c_proj_bias = tensors["h.0.attn.c_proj.bias"]

print("Attention weights (Layer 0):")
print(f"  c_attn.weight: {c_attn_weight.shape}  <- Creates Q, K, V")
print(f"  c_attn.bias:   {c_attn_bias.shape}")
print(f"  c_proj.weight: {c_proj_weight.shape}  <- Output projection")
print(f"  c_proj.bias:   {c_proj_bias.shape}")

print(f"\n💡 c_attn.weight has 2304 = 768 × 3 columns")
print(f"   First 768 cols → Q weights")
print(f"   Next 768 cols → K weights")
print(f"   Last 768 cols → V weights")


In [ ]:
# Extract Q, K, V weight matrices
n_embd = config["n_embd"]  # 768

# Split the combined weight matrix
W_q = c_attn_weight[:, :n_embd]        # [768, 768]
W_k = c_attn_weight[:, n_embd:2*n_embd]  # [768, 768]
W_v = c_attn_weight[:, 2*n_embd:]       # [768, 768]

# Split the combined bias
b_q = c_attn_bias[:n_embd]
b_k = c_attn_bias[n_embd:2*n_embd]
b_v = c_attn_bias[2*n_embd:]

print("Separated Q, K, V weight matrices:")
print(f"  W_q: {W_q.shape}")
print(f"  W_k: {W_k.shape}")
print(f"  W_v: {W_v.shape}")

print("Separated Q, K, V bias vectors:")
print(f"  b_q: {b_q.shape}")
print(f"  b_k: {b_k.shape}")
print(f"  b_v: {b_v.shape}")


---

## Computing Q, K, V: Matrix Multiplication in Action

Now we compute the Query, Key, and Value vectors for each token. Each token's embedding goes through a **linear transformation** (matrix multiplication + bias):

```
Input X: (seq_len, 768) — our token embeddings from Step 3
         ↓
┌─────────────────────────────────────────────────────────────┐
│  Q = X @ W_q + b_q    →  (seq_len, 768)  "What to search"   │
│  K = X @ W_k + b_k    →  (seq_len, 768)  "What to advertise"│
│  V = X @ W_v + b_v    →  (seq_len, 768)  "What to provide"  │
└─────────────────────────────────────────────────────────────┘
```

**What's happening:** Each 768-dimensional embedding gets transformed into three different 768-dimensional vectors. The weight matrices are learned during training to create useful Q, K, V representations.


In [ ]:
def compute_qkv(
    x: np.ndarray,       # (seq_len, n_embd) - input embeddings
    W_q: np.ndarray,     # (n_embd, n_embd) - query weights
    W_k: np.ndarray,     # (n_embd, n_embd) - key weights
    W_v: np.ndarray,     # (n_embd, n_embd) - value weights
    b_q: np.ndarray,     # (n_embd,) - query bias
    b_k: np.ndarray,     # (n_embd,) - key bias
    b_v: np.ndarray      # (n_embd,) - value bias
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Compute Query, Key, Value projections.
    
    This is like preparing database indexes:
    - Q: "What am I searching for?"
    - K: "What can I be found by?"
    - V: "What information do I contain?"
    
    Returns:
        Q, K, V each of shape (seq_len, n_embd)
    """
    # Each is a simple linear transformation: matrix multiply + bias
    Q = x @ W_q + b_q  # (seq_len, 768) @ (768, 768) + (768,) → (seq_len, 768)
    K = x @ W_k + b_k
    V = x @ W_v + b_v
    return Q, K, V


# Test with "Hello world"
text = "Hello world"
x, token_ids = get_input_embeddings(text)

Q, K, V = compute_qkv(x, W_q, W_k, W_v, b_q, b_k, b_v)

print(f"Input: '{text}'")
print(f"  x shape: {x.shape} (input embeddings)")
print(f"  Q shape: {Q.shape} (queries - 'what am I looking for?')")
print(f"  K shape: {K.shape} (keys - 'what do I contain?')")
print(f"  V shape: {V.shape} (values - 'what info do I provide?')")


In [ ]:
# Peek at the Q, K, V values
tokens = [tokenizer.get_token_string(tid).replace("Ġ", "⎵") for tid in token_ids]

print("Q, K, V values (first 5 dimensions):")
print("=" * 70)
for i, tok in enumerate(tokens):
    print(f"\nToken '{tok}' (position {i}):")
    print(f"  Q: [{', '.join(f'{v:.3f}' for v in Q[i, :5])}...]")
    print(f"  K: [{', '.join(f'{v:.3f}' for v in K[i, :5])}...]")
    print(f"  V: [{', '.join(f'{v:.3f}' for v in V[i, :5])}...]")


---

## Scaled Dot-Product Attention: The Core Algorithm

Now let's implement the attention mechanism step by step. Here's the complete pipeline:

```
┌──────────────────────────────────────────────────────────────────┐
│  ATTENTION PIPELINE                                              │
│                                                                  │
│  Step 1: Q × K^T  →  Raw match scores (how similar are they?)   │
│            ↓                                                     │
│  Step 2: ÷ √d_k   →  Scale scores (prevent explosion)           │
│            ↓                                                     │
│  Step 3: Softmax  →  Convert to percentages (load balancing)    │
│            ↓                                                     │
│  Step 4: × V      →  Weighted retrieval (get relevant info)     │
└──────────────────────────────────────────────────────────────────┘
```

Let's implement each step.

### Step 1: Q × K^T — Computing Match Scores

The dot product measures similarity between vectors. When we multiply Q with K transposed:

```
Q:   (seq_len, d_k)    — each row is a token's query
K^T: (d_k, seq_len)    — each column is a token's key
─────────────────────
Q × K^T: (seq_len, seq_len)  — scores[i,j] = how much token i matches token j
```

Think of it like computing a similarity matrix — each entry tells us how "compatible" one token's query is with another token's key.


In [ ]:
# Step 1: Compute QK^T
attention_scores = Q @ K.T  # (seq_len, seq_len)

print("Step 1: QK^T (raw attention scores)")
print(f"  Shape: {attention_scores.shape}")
print(f"\n  Score matrix (token i attending to token j):")
print(f"  Rows = query tokens, Cols = key tokens")
print()

# Pretty print the matrix
print("       ", end="")
for tok in tokens:
    print(f"{tok:>10}", end="")
print()

for i, tok in enumerate(tokens):
    print(f"{tok:>6} ", end="")
    for j in range(len(tokens)):
        print(f"{attention_scores[i, j]:>10.2f}", end="")
    print()


### Step 2: Scaling — Why Divide by √d?

**The Problem:** Look at those raw scores above — they're huge! When we pass large numbers to softmax, it "saturates" — almost all attention goes to one token (99.99%), others get nothing (0.01%).

**Why do dot products get large?**

The dot product sums up `d` component-wise multiplications. More components = larger sum (in magnitude).

For vectors with components having variance ~1:
- The **variance** of the dot product equals `d` (the dimension)
- The **standard deviation** = √d

```
Example with normalized random vectors:
───────────────────────────────────────
 Dimension   │  Std Dev of Dot Product
─────────────┼─────────────────────────
     10      │     √10  ≈  3
     64      │     √64  =  8    ← GPT-2 uses 64 dims per head
    768      │    √768  ≈ 28
───────────────────────────────────────
```

In practice, actual embeddings aren't normalized, so scores can be even larger (as we saw above: thousands!).

**The Solution:** Divide by √d_k to normalize the variance back to ~1:

```
scores = (Q @ K.T) / √d_k

Before: variance ≈ d_k  →  After: variance ≈ 1
```

This is similar to normalizing metrics before comparing them — you wouldn't compare "requests per second" directly with "latency in milliseconds" without scaling first!


In [ ]:
# For now, let's use full dimension (we'll do multi-head later)
d_k = n_embd  # 768
scale = np.sqrt(d_k)

scaled_scores = attention_scores / scale

print(f"Step 2: Scale by √d_k = √{d_k} = {scale:.2f}")
print(f"\n  Before scaling - max: {attention_scores.max():.2f}, min: {attention_scores.min():.2f}")
print(f"  After scaling  - max: {scaled_scores.max():.2f}, min: {scaled_scores.min():.2f}")


### Step 3: Softmax — Weighted Load Balancing

**Softmax is like a load balancer that distributes traffic based on server scores.**

Imagine you're building a load balancer and have these server health scores:

```
Server A: 3    (fast, healthy)
Server B: 2    (medium)
Server C: 1    (slow)
Server D: 0    (slowest)
```

You want to distribute traffic so that:
- Higher scores → more traffic
- All percentages sum to 100%

Softmax does exactly this:

```
Input scores:  [3, 2, 1, 0]   ← Server A, B, C, D
               ↓ softmax
Output %:      [64%, 24%, 9%, 3%]  ← sums to 100%!
```

**How it works:**

1. `exp(score)` — converts each score to a positive number (amplifies differences)
2. Divide each by the sum — normalizes to percentages

```python
def softmax(scores):
    exp_scores = np.exp(scores - scores.max())  # subtract max for stability
    return exp_scores / exp_scores.sum()        # divide by sum to get percentages
```

**In attention:** Softmax answers "What percentage of attention should token i pay to each other token?"


In [ ]:
def softmax(x: np.ndarray, axis: int = -1) -> np.ndarray:
    """
    Compute softmax — converts scores to percentages that sum to 100%.
    
    Think of it like a load balancer:
    - Input: server health scores [3, 2, 1, 0]
    - Output: traffic percentages [64%, 24%, 9%, 3%]
    
    The trick: subtract max before exp() to prevent numerical overflow.
    This doesn't change the result because softmax(x) = softmax(x - c).
    """
    # Subtract max for numerical stability (prevents exp() from exploding)
    x_shifted = x - x.max(axis=axis, keepdims=True)
    
    # exp() makes all values positive and amplifies differences
    exp_x = np.exp(x_shifted)
    
    # Divide by sum to get percentages
    return exp_x / exp_x.sum(axis=axis, keepdims=True)


attention_weights = softmax(scaled_scores, axis=-1)

print("Step 3: Softmax (converts to attention percentages)")
print(f"  Shape: {attention_weights.shape}")
print(f"  Each row sums to: {attention_weights.sum(axis=-1)} (100% total attention)")
print()

# Pretty print - like a load balancer traffic distribution
print("Attention weights (each row = traffic distribution for that token):")
print()
print("       ", end="")
for tok in tokens:
    print(f"{tok:>10}", end="")
print()

for i, tok in enumerate(tokens):
    print(f"{tok:>6} ", end="")
    for j in range(len(tokens)):
        weight = attention_weights[i, j]
        # Visual bar shows relative weight
        bar = "█" * int(weight * 10)
        print(f"{weight:>6.2%} {bar:<4}", end="")
    print()


### Step 4: Weighted Retrieval — Gathering Information

Now we use those attention percentages to retrieve a weighted combination of Values.

**Think of it like aggregating responses from multiple microservices:**

```
Service A returns: {"latency": 10, "status": "ok"}     weight: 0.85
Service B returns: {"latency": 50, "status": "slow"}   weight: 0.10
Service C returns: {"latency": 5, "status": "fast"}    weight: 0.05
─────────────────────────────────────────────────────────────────
Weighted result:   0.85 × A + 0.10 × B + 0.05 × C
```

In attention:
```
output = attention_weights @ V    # (seq_len, n_embd)

For each token i:
  output[i] = sum( attention_weight[i,j] × V[j] for all tokens j )
```

Each output token is now a **context-aware blend** of all token Values, weighted by how relevant each one was!


In [ ]:
# Step 4: Weighted sum of values
attention_output = attention_weights @ V

print("Step 4: Weighted sum of Values")
print(f"  attention_weights: {attention_weights.shape}")
print(f"  V:                 {V.shape}")
print(f"  output:            {attention_output.shape}")
print()
print("Each output token is now a context-aware combination of all tokens!")


---

## Putting It Together: Single-Head Attention

Let's combine all four steps into a reusable function:


In [ ]:
def attention(
    Q: np.ndarray,                    # (seq_len, d_k) - queries
    K: np.ndarray,                    # (seq_len, d_k) - keys  
    V: np.ndarray,                    # (seq_len, d_v) - values
    mask: np.ndarray = None           # (seq_len, seq_len) - optional causal mask
) -> tuple[np.ndarray, np.ndarray]:
    """
    Scaled dot-product attention — the core of transformers!
    
    Pipeline:
    1. Query-Key matching:  scores = Q × K^T
    2. Scale:              scores = scores / √d_k
    3. Mask (optional):    scores = scores + mask
    4. Load balance:       weights = softmax(scores)
    5. Weighted retrieval: output = weights × V
    
    Returns:
        output: Context-enriched representations (seq_len, d_v)
        weights: Attention distribution (seq_len, seq_len)
    """
    d_k = Q.shape[-1]  # Dimension of keys/queries
    
    # Step 1 & 2: Compute similarity scores and scale
    scores = (Q @ K.T) / np.sqrt(d_k)  # (seq_len, seq_len)
    
    # Step 3: Apply mask (for causal attention)
    if mask is not None:
        scores = scores + mask  # -inf positions become 0 after softmax
    
    # Step 4: Convert to attention percentages
    weights = softmax(scores, axis=-1)  # Each row sums to 1
    
    # Step 5: Weighted retrieval of values
    output = weights @ V  # (seq_len, d_v)
    
    return output, weights


# Test it
output, weights = attention(Q, K, V)
print(f"Attention output shape: {output.shape}")
print(f"Attention weights shape: {weights.shape}")
print(f"\nEach token now carries context from other tokens!")

---

## Quick Test: Verify Our Implementation

Let's verify our attention implementation has the expected properties:


In [ ]:
# Quiz: Verify key properties
print("🧪 Attention Concept Verification")
print("=" * 60)

# 1. Attention weights sum to 1
row_sums = weights.sum(axis=-1)  # Should all be 1.0
print(f"1. Attention weights sum to 1 per row: {np.allclose(row_sums, 1.0)} ✓")

# 2. Output dimension matches input
print(f"2. Output shape equals input shape: {x.shape == output.shape} ✓")

# 3. Q, K, V split correctly
print(f"3. QKV weight is 3x embedding dim: {c_attn_weight.shape[1] == 3 * n_embd} ✓")

print()
print("All attention properties verified! 🎉")


---

## Summary: What We Learned

### Attention = Database Query + Load Balancing

```
┌──────────────────────────────────────────────────────────────────┐
│  THE ATTENTION PIPELINE (Single-Head)                            │
├──────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Input: Token embeddings (seq_len, 768)                          │
│         ↓                                                        │
│  ┌────────────────────────────────────────────────────────────┐  │
│  │  1. QKV Projection (database query setup)                  │  │
│  │     Q, K, V = split(x @ W_qkv + b_qkv)                     │  │
│  └────────────────────────────────────────────────────────────┘  │
│         ↓                                                        │
│  ┌────────────────────────────────────────────────────────────┐  │
│  │  2. Attention:                                             │  │
│  │     a. Q × K^T          (query-key matching)               │  │
│  │     b. ÷ √768           (normalize scores)                 │  │
│  │     c. softmax          (load balance weights)             │  │
│  │     d. × V              (weighted retrieval)               │  │
│  └────────────────────────────────────────────────────────────┘  │
│         ↓                                                        │
│  Output: Context-aware embeddings (seq_len, 768)                 │
│                                                                  │
└──────────────────────────────────────────────────────────────────┘
```

### Key Concepts (with Infrastructure Analogies)

| Concept | What It Does | Analogy |
|---------|--------------|---------|
| **Q, K, V** | Query-Key-Value projections | Database: Query searches Keys, retrieves Values |
| **Dot Product** | Measures query-key similarity | Search relevance scoring |
| **Scaling (÷√d)** | Prevents score explosion | Normalizing metrics before comparison |
| **Softmax** | Converts to percentages | Load balancer distributing traffic |

---

## Next: Step 4b — Multi-Head Attention & Causal Masking

We've built single-head attention, but GPT-2 uses **12 heads** running in parallel! In **Step 4b**, we'll learn:

1. **Causal Masking** — How to prevent tokens from "seeing the future"
2. **Multi-Head Attention** — Running 12 specialized attention operations in parallel
3. **Visualization** — Seeing what different heads focus on
4. **Complete Implementation** — The full `CausalSelfAttention` class

```
┌─────────────────────────────────────────────────────────────────┐
│  MULTI-HEAD ATTENTION (Preview of Step 4b)                      │
├─────────────────────────────────────────────────────────────────┤
│  Worker 1 (Head 0):  "I focus on the previous word"            │
│  Worker 2 (Head 1):  "I track grammatical subjects"            │
│  Worker 3 (Head 2):  "I find pronoun references"               │
│  ...                                                            │
│  Worker 12 (Head 11): "I look for semantic relationships"      │
├─────────────────────────────────────────────────────────────────┤
│  Final Output: Combine all worker results                       │
└─────────────────────────────────────────────────────────────────┘
```
